In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 15994967
paper_name = 'huang_kowalski_2005' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_csv('raw_data/hit_list.txt', header=None, names=['gene'])

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 139 x 1


In [10]:
original_data.head()

,gene
0,FCY2
1,HPT1
2,NMD2
3,UPF3
4,SKI3


In [11]:
original_data['gene'] = original_data['gene'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['gene'] = clean_genename(original_data['gene'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['gene'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [gene, orf]
Index: []


In [15]:
original_data['data'] = 1

In [16]:
original_data.set_index('orf', inplace=True)

In [17]:
original_data = original_data[['data']].copy()

In [18]:
original_data = original_data.groupby(original_data.index).mean()

In [19]:
original_data.shape

(139, 1)

# Prepare the final dataset

In [32]:
data = original_data.copy()

In [33]:
dataset_ids = [183]
datasets = datasets.reindex(index=dataset_ids)

In [34]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [35]:
data.head()

dataset_id,183
data_type,value
orf,
YAL045C,1
YBR034C,1
YBR045C,1
YBR076W,1
YBR083W,1


## Subset to the genes currently in SGD

In [36]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 1


In [37]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,183
,data_type,value
gene_id,orf,
43,YAL045C,1
230,YBR034C,1
241,YBR045C,1
271,YBR076W,1
278,YBR083W,1


# Normalize

In [38]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [39]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [40]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        183          
data_type       value    valuez
gene_id orf                    
43      YAL045C     1  5.754645
230     YBR034C     1  5.754645
241     YBR045C     1  5.754645
271     YBR076W     1  5.754645
278     YBR083W     1  5.754645

# Print out

In [41]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [42]:
from IO.save_data_to_db3 import *

In [43]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 15994967...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  2.74it/s]

Updating the data_modified_on field...
